In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers,models
from sklearn.preprocessing import MinMaxScaler

df=pd.read_csv('international-airline-passengers.csv')
series=df.iloc[:,1].values.astype('float32')

scaler=MinMaxScaler()
series_scaled=scaler.fit_transform(series.reshape(-1,1)).flatten()

def create_xy(data,lookback=12):
    X,y=[],[]
    for i in range(len(data)-lookback):
        X.append(data[i:i+lookback])
        y.append(data[i+lookback])
    return np.array(X),np.array(y)

lookback=12
X,y=create_xy(series_scaled,lookback)
X=X.reshape(-1,lookback,1)

split=int(0.8*len(X))
X_train,X_test=X[:split],X[split:]
y_train,y_test=y[:split],y[split:]

model=models.Sequential([
    layers.GRU(64,input_shape=(lookback,1)),
    layers.Dense(1)
])
model.compile(optimizer='adam',loss='mse')

model.fit(X_train,y_train,epochs=100,batch_size=16,validation_data=(X_test,y_test),verbose=0)
mse=model.evaluate(X_test,y_test,verbose=0)
print("Test MSE:",mse)

pred=model.predict(X_test[:5])
pred_rescaled=scaler.inverse_transform(pred)
print("Predictions (rescaled):",pred_rescaled.flatten())

c:\Users\Mehedi Hasan\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Test MSE: nan
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 302ms/step
Predictions (rescaled): [330.0864  304.07413 338.17126 365.67242 364.04764]
